## Gradio Day
- 오늘 우리는 어이 없을 정도의 간단한 유저 인터페이스인 Gradio Framework을 사용할 것입니다

In [62]:
import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import ollama

In [29]:
import gradio as gr

In [64]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

client = ollama.Client(
    host="http://localhost:11434")



In [ ]:
openai = OpenAI()
model = 'gpt-4o-mini'

In [32]:
system_message = "You are a helpful assistant"

In [33]:
def message_gpt(prompt):
    model = 'gpt-4o-mini'
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
    ]

    completion = openai.chat.completions.create(
        model = model,
        messages=messages,
        temperature=0.7,
        max_tokens=1000,
        
    )
    # if stream is True:
    #     full_response = []
    #     for chunk in completion:
    #         full_response.append(chunk.choices[0].delta['content'])
        
    #     return full_response

    # else:
    return completion.choices[0].message.content

In [34]:
message_gpt("What is today's date")

"Today's date is October 3, 2023."

In [57]:
view = gr.Interface(
    fn = message_gpt,
    inputs = gr.Textbox(label="Enter your prompt", placeholder="Type your question here..."),
    outputs= gr.Textbox(label="Response from GPT-4o-mini"),
    title="GPT-4o-mini Chatbot",
    flagging_mode = 'never'

    
)

In [ ]:
view.launch()

In [74]:
def message_ollama(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
    ]
    response = client.chat(
        model="llama3.2",
        messages=messages,
    )
    return response['message']['content']

In [75]:
message_ollama('how are you?')

"**I'm functioning properly!**\n\nThank you for asking. I'm a large language model, so I don't have emotions or feelings like humans do, but I'm always ready to help answer your questions and provide information on a wide range of topics.\n\n**What can I assist you with today?**"

In [77]:
view = gr.Interface(
    fn = message_ollama,
    inputs = gr.Textbox(label="Enter your prompt", placeholder="Type your question here..."),
    outputs= gr.Textbox(label="Response from Ollama"),
    title="Ollama Chatbot",
    flagging_mode = 'never'

    
)

In [ ]:
view.launch()

In [92]:

# streaming gpt-4o-mini response
def stream_gpt(prompt):
    model = 'gpt-4o-mini'
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
    ]

    completion = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.7,
        max_tokens=1000,
        stream=True
    )

    # streaming once a chunk 
    full_response = ''
    for chunk in completion:
        full_response += chunk.choices[0].delta.content or ''
        if full_response:
            yield full_response
    
  

In [98]:
def stream_ollama(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
    ]
    response = client.chat(
        model="llama3.2",
        messages=messages,
        stream=True
    )

    full_response = ''
    for chunk in response:
        if 'content' in chunk['message']:
            full_response += chunk["message"]['content']
            yield full_response

In [93]:
stream_gpt('please tell me about the history of Nike')

<generator object stream_gpt at 0x151653de0>

In [ ]:
view = gr.Interface(
    fn = stream_gpt,
    inputs = gr.Textbox(label="Enter your prompt", placeholder="Type your question here..."),
    outputs= gr.Markdown(label="Response from gpt"),
    title="GPT Chatbot",
    flagging_mode = 'never'
)
view.launch()

In [99]:
view = gr.Interface(
    fn = stream_ollama,
    inputs = gr.Textbox(label="Enter your prompt", placeholder="Type your question here..."),
    outputs= gr.Markdown(label="Response from gpt"),
    title="GPT Chatbot",
    flagging_mode = 'never'
)
view.launch()

* Running on local URL:  http://127.0.0.1:7883
* To create a public link, set `share=True` in `launch()`.


In [104]:
def stream_models(prompt):
    if 'GPT':
        result = stream_gpt(prompt)
    elif 'OLLAMA':
        result = stream_ollama(prompt) 
    

In [106]:
view = gr.Interface(
    fn = stream_models,
    inputs = [gr.Textbox(label="Enter your prompt", placeholder="Type your question here..."),
              gr.Dropdown(label="Select Model", choices=["GPT", "Ollama"])
    ],
    outputs= [gr.Markdown(label='result from the model')],
    title="Stream Models Chatbot",
    flagging_mode = 'never'

)
view.launch()

/opt/anaconda3/envs/udemy_llm/lib/python3.13/site-packages/gradio/utils.py:1018: UserWarning: Expected 1 arguments for function <function stream_models at 0x151039580>, received 2.
  warnings.warn(
/opt/anaconda3/envs/udemy_llm/lib/python3.13/site-packages/gradio/utils.py:1026: UserWarning: Expected maximum 1 arguments for function <function stream_models at 0x151039580>, received 2.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7884
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/opt/anaconda3/envs/udemy_llm/lib/python3.13/site-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<5 lines>...
    )
    ^
  File "/opt/anaconda3/envs/udemy_llm/lib/python3.13/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<11 lines>...
    )
    ^
  File "/opt/anaconda3/envs/udemy_llm/lib/python3.13/site-packages/gradio/blocks.py", line 2146, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<8 lines>...
    )
    ^
  File "/opt/anaconda3/envs/udemy_llm/lib/python3.13/site-packages/gradio/blocks.py", line 1664, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^